In [1]:
import cv2
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw, ImageEnhance
import imageio
import seaborn as sns
from natsort import natsorted

## Import relevant libraries
import sys
sys.path.append('../../activedrops')
import pivdrops
pivdrops.set_plotting_style()

In [2]:
def convert_cy5_gfp_to_8bit(data_path, condition, subcondition, min_frame, max_frame):
    # Base directory where the .tif files are located
    directory = f"{data_path}/{condition}/{subcondition}/original"

    # Directories for saving the converted images
    cy5_output_directory = f"{data_path}/{condition}/{subcondition}/cy5_8bit"
    gfp_output_directory = f"{data_path}/{condition}/{subcondition}/gfp_8bit"

    # Create the output directories if they don't exist
    if not os.path.exists(cy5_output_directory):
        os.makedirs(cy5_output_directory)
    if not os.path.exists(gfp_output_directory):
        os.makedirs(gfp_output_directory)

    # Pattern to match .tif files
    search_pattern = os.path.join(directory, "img_*.tif")

    # Get a sorted list of all .tif files to ensure they are processed in order
    tif_files = sorted(glob.glob(search_pattern))[min_frame:max_frame]

    for tif_file in tif_files:
        try:
            # Read the image using OpenCV
            # img = cv2.imread(tif_file, cv2.IMREAD_UNCHANGED)
            img = imageio.imread(tif_file) / 255

            # Convert the image to 8-bit
            img_8bit = img.astype('uint8') 

            # Extract the base name
            base_name = os.path.basename(tif_file)

            # Determine the output directory based on the file name
            if "cy5" in base_name:
                output_directory = cy5_output_directory
            elif "gfp" in base_name:
                output_directory = gfp_output_directory
            else:
                continue  # Skip files that do not match 'cy5' or 'gfp'

            # Generate the new file path in the appropriate '8bit' folder
            new_file_path = os.path.join(output_directory, base_name)

            # Save the 8-bit image
            cv2.imwrite(new_file_path, img_8bit)
        except Exception as e:
            print(f"Error processing {tif_file}: {e}")


In [3]:
def plot_mean_fluorescence_over_time(data_path, condition, subcondition, channel, time_interval=3, min_frame=0, max_frame=None, skip_frames=1):
    """
    Computes and plots the mean fluorescence intensity over time for a given set of images.

    Args:
    - data_path (str): Base directory where the images are stored.
    - condition (str): Condition defining a subdirectory within the data path.
    - subcondition (str): Subcondition defining a further subdirectory.
    - channel (str): Channel specifying the fluorescence ('cy5' or 'gfp').
    """
    # Determine the directory path based on the channel
    directory_path = os.path.join(data_path, condition, subcondition, f"{channel}_8bit")
    
    # Get all .tif files in the folder
    image_files = sorted(glob.glob(os.path.join(directory_path, "*.tif")))[min_frame:max_frame:skip_frames]
    
    # Initialize a list to store results
    intensities = []
    frames = []
    
    # Loop through each image file
    for i, image_file in enumerate(image_files):
        # Read the image
        img = imageio.imread(image_file) 
        
        # Compute the mean fluorescence intensity
        mean_intensity = np.mean(img)
        
        # Append results
        frames.append(i)
        intensities.append(mean_intensity)
    
    # Create a DataFrame 
    results_df = pd.DataFrame({
        "frame": frames,
        "mean_intensity": intensities - np.min(intensities)
    })
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.plot(results_df["frame"] * time_interval / 60, results_df["mean_intensity"], marker='o', linestyle='-', color='b')
    plt.title(f"Motor expression over time \n{condition} - {subcondition} - {channel}")
    plt.xlabel("time (min)")
    plt.ylabel("Normalized Mean Fluorescence Intensity (A.U.)")
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.show()

    return results_df



In [11]:
def create_normalized_intensity_heatmap(data_path, condition, subcondition, channel, time_interval=3, min_frame=0, max_frame=None, vmax=None, skip_frames=1):
    """
    Reads each image as a matrix, creates and saves a heatmap representing the normalized pixel-wise fluorescence intensity.

    Args:
    - data_path (str): Base directory where the images are stored.
    - condition (str): Condition defining a subdirectory within the data path.
    - subcondition (str): Subcondition defining a further subdirectory.
    - channel (str): Channel specifying the fluorescence ('cy5' or 'gfp').
    - time_interval (int): Time interval in seconds between frames.
    - min_frame (int): Minimum frame number to start processing from.
    - max_frame (int, optional): Maximum frame number to stop processing at.
    """
    # Determine the directory paths based on the channel
    input_directory_path = os.path.join(data_path, condition, subcondition, f"{channel}_8bit")
    output_directory_path = os.path.join(data_path, condition, subcondition, "intensity_heatmap")
    
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_directory_path):
        os.makedirs(output_directory_path)

    # Get all .tif files in the folder
    image_files = sorted(glob.glob(os.path.join(input_directory_path, "*.tif")))[min_frame:max_frame:skip_frames] 
    
    # Loop through each image file and create a heatmap
    for i, image_file in enumerate(image_files, start=min_frame):
        # Read the image into a numpy array
        intensity_matrix = imageio.imread(image_file) 

        # Plot the heatmap
        fig, ax = plt.subplots(figsize=(8, 8))
        im = ax.imshow(intensity_matrix, cmap='gray', interpolation='nearest', extent=[-2762/2, 2762/2, -2762/2, 2762/2], vmin=0, vmax=vmax)
        plt.colorbar(im, ax=ax, label='Normalized Fluorescence Intensity (A.U.)')
        plt.title(f"Time (min): {(i - min_frame) * time_interval / 60:.2f}. \n{condition} - {subcondition} - {channel}")
        plt.xlabel('x [um]')
        plt.ylabel('y [um]')
        
        # Save the heatmap
        heatmap_filename = f"heatmap_frame_{i}.tif"
        heatmap_path = os.path.join(output_directory_path, heatmap_filename)
        plt.savefig(heatmap_path, bbox_inches='tight', pad_inches=0.1, dpi=300)
        plt.close(fig)  # Close the figure to free memory
 

In [5]:
def create_annotated_image_movies(data_path, condition, subcondition, channel, frame_rate=30, max_frame=None):
    """
    Creates video files from processed and annotated images stored in a specified directory.

    Args:
    - data_path (str): Base path where the annotated images are stored.
    - condition (str): Condition under which the annotated images are stored.
    - subcondition (str): Subcondition under which the annotated images are stored.
    - channel (str): The specific channel being processed ('cy5' or 'gfp').
    - frame_rate (int, optional): Frame rate for the output video. Defaults to 30.
    - max_frame (int, optional): Maximum number of frames to be included in the video. If None, all frames are included.
    """


    images_dir = os.path.join(data_path, condition, subcondition, "intensity_heatmap")

    image_files = natsorted(glob.glob(os.path.join(images_dir, "*.tif")))

    if max_frame is not None:
        image_files = image_files[:max_frame]

    if not image_files:
        print("No images found for video creation.")
        return

    # Get the resolution of the first image (assuming all images are the same size)
    first_image = cv2.imread(image_files[0])
    video_resolution = (first_image.shape[1], first_image.shape[0])  # Width x Height

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    out_path = os.path.join(data_path, condition, subcondition, f"{channel}_movie.avi")
    out = cv2.VideoWriter(out_path, fourcc, frame_rate, video_resolution)

    for file_path in image_files:
        img = cv2.imread(file_path)
        out.write(img)  # Write the image frame to the video

    out.release()
    print(f"Video saved to {out_path}")


In [71]:
# Example usage
data_path = "../../data/kif3_phases/"
condition = "5MT"
subcondition = "40DNA"

# convert_cy5_gfp_to_8bit(data_path, condition, subcondition, min_frame=0, max_frame=None)

In [72]:
min_frame = 0
max_frame = None

skip_frames = 8

time_interval = 30 * skip_frames
frame_rate = 120 / skip_frames


In [73]:
plot_mean_fluorescence_over_time(data_path, condition, subcondition, "cy5", time_interval=time_interval, min_frame=min_frame, max_frame=max_frame, skip_frames=skip_frames) 

In [74]:
plot_mean_fluorescence_over_time(data_path, condition, subcondition, "gfp", time_interval=time_interval, min_frame=min_frame, max_frame=max_frame, skip_frames=skip_frames)


In [75]:
create_normalized_intensity_heatmap(data_path, condition, subcondition, "cy5", time_interval=time_interval, min_frame=0, max_frame=None, vmax=62, skip_frames=skip_frames)
create_annotated_image_movies(data_path, condition, subcondition, "cy5", frame_rate=frame_rate, max_frame=None)


In [87]:
data_path = "../../data/kif3_phases/"
condition = "20MT"
subcondition = "40DNA"

create_normalized_intensity_heatmap(data_path, condition, subcondition, "gfp", time_interval=time_interval, min_frame=0, max_frame=None, vmax=4, skip_frames=skip_frames)
create_annotated_image_movies(data_path, condition, subcondition, "gfp", frame_rate=frame_rate, max_frame=None)


/var/folders/_h/zqzyskvx2hd4w984syj7bb3m0000gn/T/ipykernel_10685/2237779128.py:28: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  intensity_matrix = imageio.imread(image_file)


Video saved to ../../data/kif3_phases/20MT/40DNA/gfp_movie.avi
